In [1]:
import pandas as pd


df = pd.read_csv('prediksi-pengiriman1.csv')
df 

,Distance_km,Weather,Traffic_Level,Time_of_Day,Vehicle_Type,Preparation_Time_min,Delivery_Time_min
0,7.93,1,1,2,1,12,43
1,16.42,2,2,3,2,20,84
2,9.52,3,1,4,1,28,59
3,7.44,5,2,2,1,5,37
4,19.03,2,1,1,2,16,68
...,...,...,...,...,...,...,...
99,7.25,2,2,2,1,18,45
100,16.70,2,2,2,1,28,76
101,17.20,5,3,2,1,14,94
102,0.86,2,2,2,3,21,28


In [2]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 104 entries, 0 to 103
Data columns (total 7 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Distance_km           104 non-null    float64
 1   Weather               104 non-null    int64  
 2   Traffic_Level         104 non-null    int64  
 3   Time_of_Day           104 non-null    int64  
 4   Vehicle_Type          104 non-null    int64  
 5   Preparation_Time_min  104 non-null    int64  
 6   Delivery_Time_min     104 non-null    int64  
dtypes: float64(1), int64(6)
memory usage: 5.8 KB


In [3]:
import math
import csv
import pandas as pd


case_base_file = 'prediksi-pengiriman1.csv'

# Fungsi untuk membaca basis kasus dari file CSV
def load_case_base(file_path):
    case_base = []
    with open(file_path, mode='r') as file:
        reader = csv.DictReader(file)
        for row in reader:
            case_base.append({
                "criteria": {
                    "Distance_km": float(row["Distance_km"]),
                    "Weather": int(row["Weather"]),  
                    "Traffic_Level": int(row["Traffic_Level"]), 
                    "Time_of_Day": int(row["Time_of_Day"]), 
                    "Vehicle_Type": int(row["Vehicle_Type"]), 
                    "Preparation_Time_min": int(row["Preparation_Time_min"])
                },
                "Delivery_Time_min": int(row["Delivery_Time_min"])
            })
    return case_base

In [4]:
def criteria_to_vector(criteria):
    return [
        criteria["Distance_km"],
        criteria["Weather"],  
        criteria["Traffic_Level"],  
        criteria["Time_of_Day"],  
        criteria["Vehicle_Type"],  
        criteria["Preparation_Time_min"]
    ]

In [5]:
# Fungsi untuk menghitung Cosine Similarity
def cosine_similarity(vector1, vector2):  
    dot_product = sum(v1 * v2 for v1, v2 in zip(vector1, vector2))
    magnitude1 = math.sqrt(sum(v1 ** 2 for v1 in vector1))
    magnitude2 = math.sqrt(sum(v2 ** 2 for v2 in vector2))
    if magnitude1 == 0 or magnitude2 == 0:
        return 0
    return dot_product / (magnitude1 * magnitude2)

In [6]:
# Mencari kasus paling mirip
def find_most_similar_case(case_base, new_criteria):
    new_criteria_vector = criteria_to_vector(new_criteria)
    max_similarity = 0
    most_similar_case = None
    similarity_results = []

    for i, case in enumerate(case_base, start=1):
        case_vector = criteria_to_vector(case["criteria"])
        similarity = cosine_similarity(new_criteria_vector, case_vector)
        similarity_results.append({
            "Baris": i,
            "Criteria": case["criteria"],
            "Delivery_Time_min": case["Delivery_Time_min"],
            "Similarity": similarity
        })
        if similarity > max_similarity:
            max_similarity = similarity
            most_similar_case = case
    
    return most_similar_case, similarity_results, max_similarity

In [7]:

# Memberikan rekomendasi waktu pengiriman
def Delivery_Time_min_cbr(case_base, new_criteria):
    similar_case, similarity_results, max_similarity = find_most_similar_case(case_base, new_criteria)
    df = pd.DataFrame(similarity_results)
    df = df.sort_values(by="Similarity", ascending=False).reset_index(drop=True)
    
    # Menambahkan kolom Ranking yang dimulai dari 1
    df['Ranking'] = df.index + 1  # Ranking dimulai dari 1
    df = df[['Baris', 'Criteria', 'Delivery_Time_min', 'Similarity']] 
    

    top_10 = df.head(10)

    # Tampilkan tabel hasil similarity dan ranking
    print("\nTabel Hasil Similaritas dan Ranking (Top 10):")
    for index, row in top_10.iterrows():
        print(f"{row.name+1}. Baris: {row['Baris']} - Criteria: {row['Criteria']} - Delivery Time: {row['Delivery_Time_min']} - Similarity: {row['Similarity']}")
    
    if max_similarity >= 0.95:
        return similar_case['Delivery_Time_min']
    else:
        print("\nNilai similarity tidak memenuhi ambang batas.")
        new_Delivery_Time_min = int(input("Masukkan rekomendasi baru untuk waktu pengiriman: "))
        new_case = {
            "criteria": new_criteria,
            "Delivery_Time_min": new_Delivery_Time_min
        }
        save_case_to_csv(case_base_file, new_case)
        return new_Delivery_Time_min
# Menyimpan kasus baru ke CSV
def save_case_to_csv(file_path, new_case):
    with open(file_path, mode='a', newline='') as file:
        writer = csv.writer(file)
        if file.tell() == 0: 
            writer.writerow(["Distance_km", "Weather", "Traffic_Level", "Time_of_Day", 
                             "Vehicle_Type", "Preparation_Time_min", "Delivery_Time_min"])
        writer.writerow([new_case["criteria"]["Distance_km"],
                         new_case["criteria"]["Weather"],
                         new_case["criteria"]["Traffic_Level"],
                         new_case["criteria"]["Time_of_Day"],
                         new_case["criteria"]["Vehicle_Type"],
                         new_case["criteria"]["Preparation_Time_min"],
                         new_case["Delivery_Time_min"]])

# Penggunaan
case_base = load_case_base(case_base_file)
new_criteria = {
    "Distance_km": 5.22,
    "Weather": 2,  
    "Traffic_Level": 1, 
    "Time_of_Day": 1,  
    "Vehicle_Type": 2,  
    "Preparation_Time_min": 6
}
Delivery_Time_min = Delivery_Time_min_cbr(case_base, new_criteria)
print(f"\nPrediksi Waktu Pengiriman: {Delivery_Time_min} menit")


Tabel Hasil Similaritas dan Ranking (Top 10):
1. Baris: 90 - Criteria: {'Distance_km': 12.43, 'Weather': 4, 'Traffic_Level': 1, 'Time_of_Day': 2, 'Vehicle_Type': 3, 'Preparation_Time_min': 12} - Delivery Time: 70 - Similarity: 0.992030815153782
2. Baris: 22 - Criteria: {'Distance_km': 13.08, 'Weather': 5, 'Traffic_Level': 1, 'Time_of_Day': 3, 'Vehicle_Type': 3, 'Preparation_Time_min': 16} - Delivery Time: 58 - Similarity: 0.9920263154812045
3. Baris: 95 - Criteria: {'Distance_km': 8.32, 'Weather': 5, 'Traffic_Level': 1, 'Time_of_Day': 2, 'Vehicle_Type': 3, 'Preparation_Time_min': 12} - Delivery Time: 49 - Similarity: 0.989023458396359
4. Baris: 7 - Criteria: {'Distance_km': 9.52, 'Weather': 2, 'Traffic_Level': 1, 'Time_of_Day': 2, 'Vehicle_Type': 2, 'Preparation_Time_min': 12} - Delivery Time: 49 - Similarity: 0.9852615714355477
5. Baris: 99 - Criteria: {'Distance_km': 7.46, 'Weather': 2, 'Traffic_Level': 2, 'Time_of_Day': 3, 'Vehicle_Type': 2, 'Preparation_Time_min': 8} - Delivery Ti